In [ ]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [ ]:
import subprocess
from pathlib import Path
from collections import Counter

In [ ]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

In [ ]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8000/v1',
    '--api-key', 'xxx',
    #'--model-name', 'unsloth/gemma-3-4b-it',
    '--model-name', 'Qwen/Qwen3-4B',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    #'--input-hf-dataset', 'Nitral-AI/Reddit-NSFW-Writing_Prompts_ShareGPT',
    
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "Qwen/Qwen3-4B",
    '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)
